In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ortho_group
import os
from tqdm.notebook import tqdm
from PIL import Image
from torchvision import transforms
from torchvision.utils import save_image
import torch

In [2]:
def draw_rectangle(img, margin = None, transformation = 'NO', inside=True, size_of_rect=20):
    '''
    transformation = ['NO', 'affine', 'stretching', 'all']
    '''
    
    _a = np.array([1,  1]) / np.sqrt(2)
    _b = np.array([1, -1]) / np.sqrt(2)
    size = np.array(img.shape[:2])
    color = np.random.randint(100, 255, size=3).tolist()
    if transformation == 'NO':
        s = np.random.sample(1)[0] * 0.8 + 0.2
        a = _a * s * size_of_rect
        b = _b * s * size_of_rect
    elif transformation == 'affine':
        s = np.random.sample(1)[0] * 0.8 + 0.2
        a = _a * s * size_of_rect
        b = _b * s * size_of_rect
        
        x = ortho_group.rvs(2)
        a = a.dot(x)
        b = b.dot(x)
        print(2)
    elif transformation == 'stretching':
        s1 = np.random.sample(1)[0] * 0.8 + 0.2
        s2 = np.random.sample(1)[0] * 0.8 + 0.2
        a = ((_a + _b) * s1 + (_a - _b) * s2) * size_of_rect / 2
        b = ((_a + _b) * s1 - (_a - _b) * s2) * size_of_rect / 2
        
    elif transformation == 'affine + stretching':
        s1 = np.random.sample(1)[0] * 0.8 + 0.2
        s2 = np.random.sample(1)[0] * 0.8 + 0.2
        a = _a * s1 * size_of_rect
        b = _b * s2 * size_of_rect
        
        x = ortho_group.rvs(2)
        a = a.dot(x)
        b = b.dot(x)
        print(1)
    
    if inside:
        c = np.random.randint(np.maximum(abs(a), abs(b)), size - np.maximum(abs(a), abs(b)))
    else:
        c = np.random.randint(0, size)
        
    rect = np.expand_dims(np.vstack([c - a, c + b, c + a, c - b]), 1).astype(np.int32)
    return rect.squeeze(), cv2.drawContours(img.copy(), [rect], 0, color, 1)

In [3]:
def draw_random_curve(img, count = 1, line_size = (30, 70), variance = 0.2):
    res = img.copy()
    size = np.array(img.shape[:2])
    for _ in range(count):
        a = np.empty((np.random.randint(line_size[0], line_size[1], 1)[0], 2))
        angle = np.random.rand() * np.pi * 2
        a[0] = [np.sin(angle), np.cos(angle)]

        for i, v in enumerate(a[:-1]):
            angle = (np.random.rand() * variance + (0.5 - variance / 2)) * np.pi
            R = np.array([[np.sin(angle),-np.cos(angle)],
                          [np.cos(angle), np.sin(angle)]])
            a[i + 1] = R.dot(v)
        a = a * 1
        a = np.cumsum(a, axis=0).astype(np.int32)
        a += np.random.randint(0, size)
        
        color = np.random.randint(100, 255, size=3).tolist()
        cv2.polylines(res, [np.expand_dims(a, 1)], 0, color, 1)

    return res

In [4]:
def draw_random_triange(img, count = 1, tr_size = 10):
    res = img.copy()
    size = np.array(img.shape[:2])
    for _ in range(count):
        a = np.random.random(6).reshape(3, 2) * 2 - 1
        c = np.random.randint(0, size)
        
        rect = np.expand_dims(c + a * tr_size, 1).astype(np.int32)
        color = np.random.randint(100, 255, size=3).tolist()
    
        cv2.drawContours(res, [rect], 0, color, 1)
    return res

In [5]:
def draw_random_ellipse(img, count = 1, ell_size = 10):
    res = img.copy()
    size = np.array(img.shape[:2])
    for _ in range(count):
        a = np.random.randint(0, ell_size, 2)
        c = np.random.randint(0, size)
        
        color = np.random.randint(100, 255, size=3).tolist()
    
        cv2.ellipse(res, tuple(c), tuple(a), 0, 360, 0, color)
    return res

In [9]:
%%timeit
image = np.zeros((128, 128, 3), dtype=np.uint8)

rect, res = draw_rectangle(image, transformation = 'stretching', inside=True)

res = draw_random_curve(res, count=10)

res = draw_random_triange(res, count=10)

res = draw_random_ellipse(res, count=3)

res = Image.fromarray(res)
# plt.imshow(res)

6.08 ms ± 688 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
count_dict = dict(transformation = 'stretching', curves = 10, trianges=10, ellipses = 10)
path_name = 'data/random_0_10/'# + '_'.join([str(i) for i in count_dict.values()])

In [11]:
if not os.path.exists(path_name):
    os.makedirs(path_name)

In [12]:
t = transforms.Compose([transforms.Resize(64),
                        transforms.ToTensor(), 
                       ])

In [19]:
for epoch in tqdm(range(0, 10)):
    rect_true = np.empty((2**13, 4, 2))
    res = torch.Tensor()
    for i in tqdm(range(2**13), leave=True):
        image = np.zeros((128, 128, 3), dtype=np.uint8)
        rect, image = draw_rectangle(image, transformation = count_dict['transformation'], inside=True)
        rect_true[i] = rect
        
        counts = np.random.randint(0, 10, 3)

        image = draw_random_curve(image, count=counts[0])
        image = draw_random_triange(image, count=counts[1])
        image = draw_random_ellipse(image, count=counts[2])

        image = Image.fromarray(image)
        image = t(image)
        res = torch.cat([image.unsqueeze(0), res], 0)

#         plt.imsave(f'{path_name}/image_{i}.png', image)
    torch.save(res, f'{path_name}/{str(epoch).zfill(2)}')
    np.save(f'{path_name}/{str(epoch).zfill(2)}', rect_true)
        
    del res, rect_true